In [1]:
from utils import set_env
import time
set_env()
from datasets.arrow_dataset import Dataset
import pyarrow as pa
i1=time.time()
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
i2=time.time()
print('1', i2-i1)
from datasets import load_dataset
i3=time.time()
print('2',i3-i2)
import torch
i4=time.time()
print('3', i4-i3)

from trl import SFTTrainer
i5=time.time()
print('4',i5-i4)
from peft import LoraConfig, PeftModel
from trl import SFTConfig

1 3.5864217281341553
2 0.0001590251922607422
3 4.8160552978515625e-05


2024-12-03 20:32:32.724045: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733279552.844690 3179527 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733279552.848383 3179527 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


4 8.459800720214844


In [2]:
base_model_id = "meta-llama/Llama-3.2-1B"

model_original = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_eos_token=True,
    add_bos_token=True, 
)
tokenizer.pad_token = tokenizer.eos_token

In [25]:

# dataset_name = "scooterman/guanaco-llama3-1k"
train, test= torch.load('codes/src/trainset_llm.pkl', weights_only=True), torch.load('codes/src/testset_llm.pkl', weights_only=True)
for i in range(len(train)):
    train[i]['prompt_all_text']=f"{train[i]['prompt_all_text']} {'good' if train[i]['label']==0 else 'bad'}"
train=Dataset(pa.Table.from_pylist([{'text':x['prompt_all_text']} for x in train]))
# test=Dataset(pa.Table.from_pylist([{'text':x['prompt_all_text']} for x in test]))

In [ ]:
# Output directory where the results and checkpoint are stored
import copy


output_dir = "./results"
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03
# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True
# Save checkpoint every X updates steps
save_steps = 100
logging_steps = 5
training_arguments = SFTConfig(
    max_seq_length=1024,
    output_dir=output_dir,
    num_train_epochs=2,
    per_device_train_batch_size=10,
    gradient_accumulation_steps=2,
    optim=optim,
    save_steps=100,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    bf16=True,
    max_grad_norm=max_grad_norm,
    max_steps=-1,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    dataset_text_field="text",
    report_to=None
)
model=copy.deepcopy(model_original)
trainer = SFTTrainer(
    model=model,
    train_dataset=train,
    tokenizer=tokenizer,
    args=training_arguments,
)
trainer.train()

In [32]:
from transformers import pipeline
pipe = pipeline(task='text-generation', model=model, tokenizer=tokenizer,
    stop_strings=["\n",'.'],max_new_tokens=2000)

In [45]:
from tqdm import trange


correct=0
res1=[]
for i in trange(len(test)):
    res=pipe(test[i]['prompt_all_text'], tokenizer=tokenizer)[0]['generated_text']
    res_1=res[len(test[i]['prompt_all_text']):].strip()
    res1.append(res_1)
    guess=0 if res_1=='good' else 1
    correct+=guess==test[i]['label']

100%|██████████| 354/354 [00:24<00:00, 14.65it/s]


In [46]:
import pandas as pd


pd.Series(res1).apply(lambda x:x.strip()).value_counts()

good    230
bad     124
Name: count, dtype: int64

In [48]:
correct/len(test)

tensor(0.9944)